In [1]:
import pandas as pd
import numpy as np
import os
import gget
import sklearn
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
import sys
import itertools
import scipy
import scanpy as sc

import utils as ut

In [2]:
fpath = "/nfs/turbo/umms-indikar/shared/projects/spatial_transcriptomics/data/WATAtlas/metadata.tsv.gz"

mdf = pd.read_csv(fpath, sep="\t")
print(f"{mdf.shape=}")

# # drop human samples
mdf = mdf[mdf['species__ontology_label'] == 'Mus musculus']
mdf = mdf[mdf['depot__ontology_label'] == 'epididymal fat pad']
print(f"{mdf.shape=}")

mdf['cell_type__ontology_label'].value_counts()

# mdf.head()



/sw/pkgs/arc/python3.9-anaconda/2021.11/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3172: DtypeWarning: Columns (19,20,21,22,23,24,30,32,33,36,37,39,40,41) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


mdf.shape=(363870, 58)
mdf.shape=(71780, 58)


macrophage                              35876
preadipocyte                            10945
fat cell                                 7620
mesothelial cell                         6542
epithelial cell                          3499
monocyte                                 2895
endothelial cell                         1366
dendritic cell                            734
endothelial cell of lymphatic vessel      558
T cell                                    498
B cell                                    490
mast cell                                 269
immature NK T cell                        228
pericyte cell                             168
smooth muscle cell                         56
neutrophil                                 29
mammary gland epithelial cell               7
Name: cell_type__ontology_label, dtype: int64

In [3]:
# # get the preadipocyte cell type
# adf = mdf[mdf['cell_type__ontology_label'] == 'preadipocyte']
# print(f"{adf.shape=}")

# load the expression data

In [4]:
mtxPath = "/nfs/turbo/umms-indikar/shared/projects/spatial_transcriptomics/data/WATAtlas/"
data = sc.read_10x_mtx(mtxPath, cache=True)
data.var_names = [x.upper() for x in data.var_names] # uppercase gene names
print('done loading')

# convert to dataframe
df = data.to_df()
print(df.shape)
df.head()

done loading
(197721, 25180)


0610005C13RIK  0610006L08RIK  0610009B22RIK  \
Mm_ING_16-1_TCAAGCACACAGCGCT            0.0            0.0            6.0   
Mm_ING_16-1_CCTCCAAGTATGGAAT            0.0            0.0            0.0   
Mm_ING_16-1_TTCTTCCAGCAAGCCA            0.0            0.0            0.0   
Mm_ING_16-1_CACCGTTTCCGCTGTT            0.0            0.0            0.0   
Mm_ING_16-1_AAACGCTCAACCTATG            0.0            0.0            3.0   

                              0610009E02RIK  0610009O20RIK  0610010F05RIK  \
Mm_ING_16-1_TCAAGCACACAGCGCT            1.0            5.0            8.0   
Mm_ING_16-1_CCTCCAAGTATGGAAT            1.0            2.0            5.0   
Mm_ING_16-1_TTCTTCCAGCAAGCCA            1.0            3.0            4.0   
Mm_ING_16-1_CACCGTTTCCGCTGTT            1.0            2.0           15.0   
Mm_ING_16-1_AAACGCTCAACCTATG            0.0            3.0            3.0   

                              0610010K14RIK  0610012D04RIK  0610012G03RIK  \
Mm_ING_16-1_TCAAGCACACAGCGCT            0.0            0.0            4.0   
Mm_ING_16-1_CCTCCAAGTATGGAAT            0.0            0.0            5.0   
Mm_ING_16-1_TTCTTCCAGCAAGCCA            0.0            0.0            0.0   
Mm_ING_16-1_CACCGTTTCCGCTGTT            0.0            0.0            1.0   
Mm_ING_16-1_AAACGCTCAACCTATG            0.0            0.0            2.0   

                              0610025J13RIK  ...  GM45843  PRM2  WFDC16  \
Mm_ING_16-1_TCAAGCACACAGCGCT            0.0  ...      0.0   0.0     0.0   
Mm_ING_16-1_CCTCCAAGTATGGAAT            0.0  ...      0.0   0.0     0.0   
Mm_ING_16-1_TTCTTCCAGCAAGCCA            0.0  ...      0.0   0.0     0.0   
Mm_ING_16-1_CACCGTTTCCGCTGTT            0.0  ...      0.0   0.0     0.0   
Mm_ING_16-1_AAACGCTCAACCTATG            0.0  ...      0.0   0.0     0.0   

                              ZFP648  CMTM2A  GM14967  GM15156  RNASE2B  \
Mm_ING_16-1_TCAAGCACACAGCGCT     0.0     0.0      0.0      0.0      0.0   
Mm_ING_16-1_CCTCCAAGTATGGAAT     0.0     0.0      0.0      0.0      0.0   
Mm_ING_16-1_TTCTTCCAGCAAGCCA     0.0     0.0      0.0      0.0      0.0   
Mm_ING_16-1_CACCGTTTCCGCTGTT     0.0     0.0      0.0      0.0      0.0   
Mm_ING_16-1_AAACGCTCAACCTATG     0.0     0.0      0.0      0.0      0.0   

                              GM12343  GM28231  
Mm_ING_16-1_TCAAGCACACAGCGCT      0.0      0.0  
Mm_ING_16-1_CCTCCAAGTATGGAAT      0.0      0.0  
Mm_ING_16-1_TTCTTCCAGCAAGCCA      0.0      0.0  
Mm_ING_16-1_CACCGTTTCCGCTGTT      0.0      0.0  
Mm_ING_16-1_AAACGCTCAACCTATG      0.0      0.0  

[5 rows x 25180 columns]

In [5]:
# get list of genes expressed in our data
fpath =  "/nfs/turbo/umms-indikar/shared/projects/spatial_transcriptomics/data/CARDInputs/HFD14_global_spt.csv"
sdf = pd.read_csv(fpath)
sdf = sdf.rename(columns={'Unnamed: 0' : 'gene'})
genes = sdf['gene'].to_list()
genes[0:10]

# filter genes
expGenes = [x for x in df.columns if x in genes]
print(len(expGenes))

"""WARNING - changing the raw data structure... """
df = df[expGenes]
print(df.shape)

23313
(197721, 23313)


In [17]:
def squeeze_dataframe(df):
    cols = dict(df.dtypes)
    for col, type in cols.items():
        if type == 'float64':
            df[col] = pd.to_numeric(df[col], downcast='float')
        elif type == 'int64':
            df[col] = pd.to_numeric(df[col], downcast='integer')
        elif type == 'object':
            df[col] = df[col].astype(CategoricalDtype(ordered=True))

    return df

In [21]:
# save the individual files
saveDir = "/nfs/turbo/umms-indikar/shared/projects/spatial_transcriptomics/data/WATAtlas/subtypes/"

for cellType in mdf['cell_type__ontology_label'].unique():
    print()
    meta = mdf[mdf['cell_type__ontology_label'] == cellType]
    cids = meta['cell_id'].to_list()
    print(f"{cellType} {len(cids)=}")
    
    cx = df[df.index.isin(cids)]
    cx = squeeze_dataframe(cx)
    print(f"{cx.shape=}")
    
    cellName = "_".join(cellType.split(" "))
    
    # save the expression
    savePath = f"{saveDir}{cellName}_rna.parquet"
    cx.to_parquet(savePath, index=True)
    print(f"saved {cellName} expression")
    
    # save the metadata
    savePath = f"{saveDir}{cellName}_labels.parquet"
    meta.to_parquet(savePath, index=True)
    print(f"saved {cellName} metadata")
    
    
print('done')


macrophage len(cids)=35876
cx.shape=(35876, 23313)
saved macrophage expression
saved macrophage metadata

preadipocyte len(cids)=10945
cx.shape=(10945, 23313)
saved preadipocyte expression
saved preadipocyte metadata

B cell len(cids)=490
cx.shape=(490, 23313)
saved B_cell expression
saved B_cell metadata

monocyte len(cids)=2895
cx.shape=(2895, 23313)
saved monocyte expression
saved monocyte metadata

immature NK T cell len(cids)=228
cx.shape=(228, 23313)
saved immature_NK_T_cell expression
saved immature_NK_T_cell metadata

endothelial cell len(cids)=1366
cx.shape=(1366, 23313)
saved endothelial_cell expression
saved endothelial_cell metadata

mesothelial cell len(cids)=6542
cx.shape=(6542, 23313)
saved mesothelial_cell expression
saved mesothelial_cell metadata

fat cell len(cids)=7620
cx.shape=(7620, 23313)
saved fat_cell expression
saved fat_cell metadata

dendritic cell len(cids)=734
cx.shape=(734, 23313)
saved dendritic_cell expression
saved dendritic_cell metadata

pericyte ce

In [10]:
mdf['cell_id'].head()

166149    Mm_EPI_10-1_AAACCCAAGAAGGCTC
166150    Mm_EPI_10-1_AAACCCAAGGGCAGAG
166151    Mm_EPI_10-1_AAACCCAGTAGCCCTG
166152    Mm_EPI_10-1_AAACCCAGTCTCTCTG
166153    Mm_EPI_10-1_AAACCCAGTTTCAGAC
Name: cell_id, dtype: object